In [43]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from scipy.stats import pearsonr

In [345]:
# convert RGB to normalized intensity
def rgb_to_intensity(img_path):
    img_rgb = mpimg.imread(img_path)
    x, y, _ = img_rgb.shape

    img_intensity = np.zeros((x, y))
    for i in range(x):
        for j in range(y):
            r, g, b = img_rgb[i][j]
            img_intensity[i][j] = (0xFFFF * r + 0xFF * g + b) \
                                    / (0xFFFF + 0xFF + 1) / 0xFF
    return img_intensity

In [63]:
# find the maximum feature correlation
# allowing certain ranges of x, y offset
# the feature sizes are fixed and the same
def find_xy_of_max_corr(file_path):
    
    img = rgb_to_intensity(file_path)
    
    dx, dy = 40, 125 
    x10, y10 = 10, 2

    device_1 = img[y10:y10 + dy, x10:x10 + dx]
    
    res = []
    for delta_x in range(72, 79):
        for delta_y in range(-2, 1):
            x20, y20 = x10 + delta_x, y10 + delta_y
            device_2 = img[y20:y20 + dy, x20:x20 + dx]
            
            corr_coef = pearsonr(device_1.ravel(), 
                                 device_2.ravel())[0]
            res.append([delta_x, delta_y, corr_coef])

    res.sort(key = lambda x: -x[2])
    
    return res[0]

In [332]:
folder = r'..\Extracted_Images'.replace('\\', '\\')
file_paths = [os.path.join(folder, file_dir) \
              for file_dir in os.listdir(folder)]

In [111]:
corrs = []
for img_file in file_paths:
    corrs.append(find_xy_of_max_corr(img_file))

ave_sig = [np.mean(rgb_to_intensity(file_path)) \
           for file_path in file_paths]

bias = [4/484 * i -2 for i in range(485)]
x = bias[225:350]

fig, ax1 = plt.subplots()
fig.set_size_inches(12, 3.5)
ax2 = ax1.twinx()

ax1.plot(x, corr_array[225:350], 
         color = 'b', label = 'Correlation')

ax2.plot(x, [sig for sig in ave_sig[225:350]], 
         color = 'r', label = 'Average Signal')

ax1.set_xlabel('Sample Bias (V)')
ax1.set_ylabel('Correlation (a.u.)', color='b')
ax2.set_ylabel('Average Signal (a.u.)', color='r')

fig.legend(loc=(0.75, 0.2))

fig.savefig('Corr_intensity.png', dpi = 2**7)

In [ ]:
from collections import Counter
cors_count = Counter([(x, y) for x, y, _ in corrs])
cors_count

In [333]:
corrs  # the max corrrelation at offset are 428, 88.4% [76, -2]
       # 44, 9.09% [75, -2]
       # 8, 1.65% [76, 1]
       # 4, 0.82% [76, 0]

[[76, -2, 0.11324821148116414],
 [76, -2, 0.11324821148116414],
 [76, -2, 0.11324821148116414],
 [76, -2, 0.11324821148116414],
 [76, -2, 0.11324821148116414],
 [76, -2, 0.11324821148116414],
 [76, -2, 0.11962659217413503],
 [76, -2, 0.11962659217413503],
 [76, -2, 0.11962659217413503],
 [76, -2, 0.11962659217413503],
 [76, -2, 0.22479004152623425],
 [76, -2, 0.22479004152623425],
 [76, -2, 0.22479004152623425],
 [76, -2, 0.22479004152623425],
 [76, -2, 0.45035625346331054],
 [76, -2, 0.45035625346331054],
 [76, -2, 0.45035625346331054],
 [76, -2, 0.45035625346331054],
 [76, -2, 0.6362502108036362],
 [76, -2, 0.6362502108036362],
 [76, -2, 0.6362502108036362],
 [76, -2, 0.6362502108036362],
 [76, -2, 0.7569237928491347],
 [76, -2, 0.7569237928491347],
 [76, -2, 0.7569237928491347],
 [76, -2, 0.7569237928491347],
 [76, -2, 0.820682070456094],
 [76, -2, 0.820682070456094],
 [76, -2, 0.820682070456094],
 [76, -2, 0.820682070456094],
 [76, -2, 0.8468525350084045],
 [76, -2, 0.8468525350084

In [308]:
kwargs = dict(origin = 'upper',
              cmap = 'winter',
              vmin = 0,
              vmax = 1,
              extent = [0, 3, 0, 3]
              )

for i in [225, 285, 345]:
    img = rgb_to_intensity(file_paths[i])
    
    plt.imshow(img, **kwargs)
    
    plt.annotate(' (Cor, Sig)\n\n(%0.2f, %0.2f)'%(corr_array[i], 
                                                  np.mean(img)), 
                 xy = (1.1, 1.3), color = 'white', weight='bold')
    
    plt.colorbar(shrink = 0.5, ticks = [0.0, 0.5, 1.0])
    plt.xticks([0, 1, 2, 3])
    plt.yticks([0, 1, 2, 3])
    plt.xlabel(r'$\mu$m')
    plt.ylabel(r'$\mu$m')
    plt.title('Signal at Bias: %0.2f V'%(bias[i]))
    
    plt.savefig('%d.png'%(i), dpi = 2**7)
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [56]:
res

[[76, -2, 0.8961645699299449],
 [76, -1, 0.8726012498750876],
 [75, -2, 0.8640150686172584],
 [77, -2, 0.8544312295337096],
 [75, -1, 0.840117823972009],
 [77, -1, 0.8370848120730717],
 [76, 0, 0.8359035219200365],
 [77, 0, 0.8032950780743499],
 [75, 0, 0.8023655720899401],
 [74, -2, 0.7801751642170811],
 [78, -2, 0.7728213916970096],
 [74, -1, 0.7586565201581248],
 [78, -1, 0.7577242799712961],
 [78, 0, 0.730559443208231],
 [74, 0, 0.7236579419054596],
 [73, -2, 0.6867764084623419],
 [73, -1, 0.6674737346545803],
 [73, 0, 0.6379150699273601],
 [72, -2, 0.5954618265538143],
 [72, -1, 0.578939175840673],
 [72, 0, 0.5503544765845042]]